- get chatgpt to accept prompts
- get chatgpt to output prompts

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [33]:
import os
import openai
from google.colab import userdata
import pandas as pd
import re

openai.api_key = userdata.get('API_KEY')
MODEL = "gpt-3.5-turbo-16k"

In [4]:
#Initial call to GPT to understand the trials
init_sys_call = [
    {"role": "system",
    "content": """Your task is to give judgement about where you were based on the time we give. You will give 4 alternative options. You MUST select one.

                  Example:

                  Presented:
                  1 at 6PM 11/08/2019
                  2 at 7PM 12/08/2019
                  ...

                  Question:
                  Where were you at 6PM 11/09/2019:
                  A. 2
                  B. 3
                  C. 1
                  D. 30

                  Response:
                  C

                  Where were you at 7PM 12/09/2019:
                  A. 2
                  B. 3
                  C. 9
                  D. 30

                  Response:
                  A
                  ..."""
              }]

In [6]:
events = pd.read_csv(userdata.get('EVENTS_FILE'))
experiments = pd.read_csv(userdata.get('EXPERIMENTS_FILE'))

# # Base Case Events - 1st User's ID, StartDateTime UTC, GPS Cluster Original
# USER_ID = "ap-northeast-1:2e3dfd5b-19ff-4b99-abe5-bbee4d46749f"
# events_1 = events[['ID', 'StartDateTime Local', 'GPS Cluster Original']][events['USER ID']==USER_ID]


# # Base Case Experiment - 1st ID, Target, A, B, C, D
# experiments_1 = experiments[['ID', 'Target', 'A', 'B', 'C', 'D']][experiments['USER ID']==USER_ID]

# # offsets
# events_1_offset = events_1.index.min()
# experiments_1_offset = experiments_1.index.min()
# experiments_1


In [22]:
# Helpful constants
EVENT_DF = 0
EVENT_OFFSET = 1
EXPERIMENT_DF = 2
EXPERIMENT_OFFSET = 3


def create_event_experiment_dfs(uuid):
  """
  create_event_experiment_dfs() creates the appropriate dataframes and index offsets to reference events and experiments.

  :param USER_ID: the user's unique identification string
  :return: user's events, event offset index, user's experiment, experiment offset index
  """

  events_temp = events[['ID', 'StartDateTime Local', 'GPS Cluster Original']][events['USER ID']==uuid]
  experiments_temp = experiments[['ID', 'Target', 'A', 'B', 'C', 'D']][experiments['USER ID']==uuid]

  # offsets
  events_offset = events_temp.index.min()
  experiments_offset = experiments_temp.index.min()
  return (events_temp, events_offset, experiments_temp, experiments_offset)

temp = create_event_experiment_dfs("ap-northeast-1:e7c916fc-736e-47e6-979a-c1c937ebe094")


In [23]:
# Let GPT know where it was

def gen_location(Event_Exp_df):
  """
  gen_location() generates the location information of the given data
  """

  events_df = Event_Exp_df[EVENT_DF]
  events_offset = Event_Exp_df[EVENT_OFFSET]

  location_data = []
  for i in range(len(events_df)):
    datetime = events_df.loc[i+events_offset, "StartDateTime Local"]
    date_pattern = r"(\d{4}-\d{2}-\d{2})T"
    time_pattern = r"\d{4}-\d{2}-\d{2}T(\d{2}:\d{2}:\d{2})Z"
    date = re.search(date_pattern, datetime).group(1)
    time = re.search(time_pattern, datetime).group(1)

    gps_cluster = events_df.loc[i+events_offset, "GPS Cluster Original"]
    location_data.append({
        "role": "user",
        "content": f"{gps_cluster} at {time} {date}"
    })

  return location_data




  # 2019-08-11T06:00:00Z
  #  print(events_1.loc[i, "ID"], events_1.loc[i, "GPS Cluster Original"])
temp_loc = gen_location(temp)

In [24]:
# Form Questions for GPT

def gen_questions(Event_Exp_df):

  events_df = Event_Exp_df[EVENT_DF]
  experiments_df = Event_Exp_df[EXPERIMENT_DF]
  experiments_offset = Event_Exp_df[EXPERIMENT_OFFSET]

  question_data = []
  for i in range(len(experiments_df)):
    question = ""
    response_id = experiments_df.loc[i+experiments_offset, "Target"]
    correct_event_id = experiments_df.loc[i+experiments_offset, response_id]

    # Cluster for A, B, C, D respectively
    cluster_question = []
    for alpha in ['A', 'B', 'C', 'D']:
      alpha_response_id = experiments_df.loc[i+experiments_offset, alpha]
      cluster_question.append(events_df.loc[alpha_response_id, "GPS Cluster Original"])

    datetime = events_df.loc[correct_event_id, "StartDateTime Local"]
    date_pattern = r"(\d{4}-\d{2}-\d{2})T"
    time_pattern = r"\d{4}-\d{2}-\d{2}T(\d{2}:\d{2}:\d{2})Z"
    date = re.search(date_pattern, datetime).group(1)
    time = re.search(time_pattern, datetime).group(1)

    question += f"Where were you at {time} {date}:\n"
    question += f"A. {cluster_question[0]}\n"
    question += f"B. {cluster_question[1]}\n"
    question += f"C. {cluster_question[2]}\n"
    question += f"D. {cluster_question[3]}"

    question_data.append({
        "role": "user",
        "content": question
    })

  return question_data

temp_ques = gen_questions(temp)

In [27]:

def run_gpt(Event_Exp_df, location_data, question_data):

  experiments_df = Event_Exp_df[EXPERIMENT_DF]
  output_msg = []
  message = {}

  for i in range(len(experiments_df)):
    message = init_sys_call + location_data

    if (i==0):
      message.append(question_data[0])
    else:
      for j in range(i+1):
        message.append(question_data[j])
        if (j<i):
          message.append({'role': 'assistant', 'content': output_msg[j]})

    # output_msg.append('A')
    # print(message)
    response = openai.chat.completions.create(
      model=MODEL,
      messages=message
    )

    # print(response)

    if (response['choices'][0]['finish_reason'] == "length"):
      print("ERROR: Too Long")
      break

    output_msg.append(response['choices'][0]['message']['content'])

  return output_msg



In [ ]:
len(output_msg)

27

In [ ]:
response

<OpenAIObject chat.completion id=chatcmpl-8ABJAQupInduw6mDMTI5bUmAkALhZ at 0x7bcc5275d8a0> JSON: {
  "id": "chatcmpl-8ABJAQupInduw6mDMTI5bUmAkALhZ",
  "object": "chat.completion",
  "created": 1697437440,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "B"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 10738,
    "completion_tokens": 1,
    "total_tokens": 10739
  }
}

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(output_msg, columns=["Response"])
df.to_csv('/content/data/responseFULLparticipant7.csv', index=True)

In [34]:
output = run_gpt(temp, temp_loc, temp_ques)
output

TypeError: ignored

# TODOs
- [x] Change all the code into functions
- [ ] Migrate code for new version of GPT API
- [ ] Amend all temporal information to be more human friendly

# Prev Minutes
- compare human against gpt
- gpt not nearly as accurate as human

# Model tuning
- Question & content:
  - convert dates to day (frame questions in terms on weeks)
  - x weeks ago, on monday at this time
  - change time to e.g., 6am
  - change cluster to location 0 (and include the word in questions) (sidebar - diff between place & location)

- sequential dependencies: what happens if you dont give it the prev questions
  - llms have tendency to repeat itself

- Assume midnight observations are at home

Then
- clustering:
  - clusters don't seem like locations
  - can we get gps to decode?
    - pro: semantically meaningful name insteaad of a cluster num
    - cons: error in conversion since it's not accurate


